In [142]:
%matplotlib inline

import numpy as np
from skimage import io, filters, segmentation
import matplotlib.pyplot as plt
import os
import glob
from scipy.cluster.vq import kmeans2, kmeans, vq, whiten

if not os.path.exists("Q3"):
    os.makedirs("Q3")

In [149]:
if __name__ == '__main__':
    path = 'Images/Q3'
    fig = plt.figure()
    count = 1
    for filename in glob.glob(os.path.join(path, '*.bmp')): 
        a = fig.add_subplot(2, 2, count)
        im = plt.imread(filename)
        imgplot = plt.imshow(im, cmap=plt.get_cmap('gray'), vmin=0, vmax=255)
        plt.axis('off')
        plt.savefig('Q3/allimgs.png', dpi=200)
        count += 1
    plt.close()

In [150]:
# calculate minimum padding (one side of the image)
def getSmallPad(MaskDim):
    pad = int(MaskDim / 2) - (1 - (MaskDim % 2))
    if (pad < 0):
        pad = 0
    return pad


# calculate total dimensional padding (both sides)
def getTotalPad(MaskDim, ImgDim):
    smallPad = getSmallPad(MaskDim)
    bigPad = MaskDim / 2
    return ImgDim + smallPad + bigPad, smallPad, bigPad


# fetch the neighbors for a particular value in a matrix
def fetch_neighbors(row, col, padded, ker_w, ker_h):
    neighbors = padded[row:row + ker_w, col:col + ker_h]
    return neighbors


# runs a matrix operation on a large image based on the provided fun function
# by default will perform the flips expected by convolution
# implementation of zero pad boundary handling
def handle_window_hist_vector(angles, mags, size_y, size_x, n_feats, thresh):
    result = np.empty(angles.shape + (n_feats,))

    total_width, l_pad, r_pad = getTotalPad(size_x, angles.shape[1])
    total_height, t_pad, b_pad = getTotalPad(size_y, angles.shape[0])
    
    padded_thresh = np.zeros((total_width, total_height), dtype=np.float)
    # padding angles probably not needed but not sure how to deal
    padded_angles = np.zeros((total_width, total_height), dtype=np.float)

    padded_thresh[l_pad:-r_pad, t_pad:-b_pad] = mags
    padded_angles[l_pad:-r_pad, t_pad:-b_pad] = angles

    for (row, col), value in np.ndenumerate(angles):
        neighbors_t = fetch_neighbors(row, col, padded_thresh, size_x, size_y)
        neighbors_a = fetch_neighbors(row, col, padded_angles, size_x, size_y)
        hist_vals = neighbors_a[np.where(neighbors_t > thresh)]
        result[row, col] = np.histogram(hist_vals, n_feats)[0]
    
    return result

In [153]:
if __name__ == '__main__':
    path = 'Images/Q3'
    ks = [5, 10, 50]
    for filename in glob.glob(os.path.join(path, '*.bmp')): 
        im = plt.imread(filename)

        s_h = filters.sobel_h(im)
        print 'Mx'
        print s_h
        s_v = filters.sobel_v(im)
        print 'My'
        print s_v
        mag = np.sqrt(s_h**2 + s_v**2)
        print 'M'
        print mag
        direc = np.arctan2(s_v, s_h)
        print 'Theta'
        print direc
            
        scaled_dir = np.copy(direc)
        # rounding
        eighth = 2*np.pi/8
        scaled_dir = np.round(scaled_dir/eighth)*eighth
        # account for cyclical -- -pi to pi
        scaled_dir[scaled_dir == -np.pi] = np.pi

        vectorized = handle_window_hist_vector(scaled_dir, mag, 11, 11, 8, 0.1)
        vectorized = np.reshape(vectorized, (vectorized.shape[0] * vectorized.shape[1], vectorized.shape[2]))
        
        whitened = whiten(vectorized)
        filen = filename.split('/')[-1].split('.')[0]
        for k in ks:
            codebook, _ = kmeans2(whitened, k)
            quantz, _ = vq(vectorized, codebook)
            cent_id = np.reshape(quantz, im.shape)

            fn = filen + '_k' + str(k) + '.png'
            fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(10, 10))
            heatmap = ax.imshow(cent_id, vmin=cent_id.min(), vmax=cent_id.max()) 
            plt.axis('off')
            plt.savefig('Q3/' + fn, dpi=200)
            plt.close()

Mx
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.01176471  0.02058824 ..., -0.02156863 -0.01078431  0.        ]
 [ 0.          0.03235294  0.01372549 ...,  0.0127451   0.00392157  0.        ]
 ..., 
 [ 0.          0.23921569  0.3745098  ...,  0.02058824  0.07745098  0.        ]
 [ 0.         -0.23529412 -0.22843137 ...,  0.17254902  0.13529412  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
My
[[ 0.          0.          0.         ...,  0.          0.          0.        ]
 [ 0.          0.02352941  0.00686275 ...,  0.02156863  0.01078431  0.        ]
 [ 0.          0.03235294 -0.01176471 ...,  0.00882353  0.0254902   0.        ]
 ..., 
 [ 0.          0.22352941 -0.06666667 ...,  0.12058824  0.00686275  0.        ]
 [ 0.          0.32156863 -0.02254902 ...,  0.13137255  0.01568627  0.        ]
 [ 0.          0.          0.         ...,  0.          0.          0.        ]]
M
[[ 0.          0